In [207]:
import requests
from bs4 import BeautifulSoup

url = "https://www.pokepedia.fr/Liste_des_Pokémon_de_la_neuvième_génération"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table", class_="tableaustandard")

In [208]:
if table:
    pokemon_names = []
    pokemon_types = []
    
    rows = table.find_all("tr")[1:]  # On saute l'en-tête

    for row in rows:
        cells = row.find_all("td")

        if len(cells) >= 8:  # Vérifie qu'il y a au moins 8 colonnes pour éviter l'erreur
            # 🟢 Récupération du nom du Pokémon
            name_tag = cells[2].find("a") if cells[2].find("a") else None
            pokemon_names.append(name_tag.text.strip() if name_tag else "Inconnu")  # Ajoute "Inconnu" si absent
            
            # 🟢 Récupération des types du Pokémon
            types_tags = cells[7].find_all("span") if cells[7] else []  # Évite une erreur si cells[7] est absent
            types = []
            for type_tag in types_tags:
                a_tag = type_tag.find("a")
                if a_tag and "title" in a_tag.attrs:
                    types.append(a_tag['title'].replace(" (type)", ""))

            # 🛠 Assurer un format uniforme (toujours une liste)
            pokemon_types.append(types if types else ["Inconnu"])  # Ajoute ["Inconnu"] si aucun type trouvé


mot_recherche = "Inconnu"

if mot_recherche in pokemon_types:
    index = liste.index(mot_recherche)
    print(f"Le mot '{mot_recherche}' est à l'index {index}.")
else:
    print(f"Le mot '{mot_recherche}' n'est pas dans la liste.")


Le mot 'Inconnu' n'est pas dans la liste.


In [209]:
#Récuperer les noms en anglais : 
pokemon_names_anglais = []

rows = table.find_all("tr")[1:]  # On saute la première ligne qui contient les en-têtes

for row in rows:
        cells = row.find_all("td")

        if len(cells) >= 8:  # Vérifie qu'il y a au moins 8 colonnes pour éviter l'erreur
            # 🟢 Récupération du nom du Pokémon
            name_tag = cells[3].find("a") if cells[3].find("a") else None
            if name_tag.text not in pokemon_names_anglais:
                pokemon_names_anglais.append(name_tag.text.strip() if name_tag else "Inconnu")  # Ajoute "Inconnu" si absent

pokemon_names_anglais

['Sprigatito',
 'Floragato',
 'Meowscarada',
 'Fuecoco',
 'Crocalor',
 'Skeledirge',
 'Quaxly',
 'Quaxwell',
 'Quaquaval',
 'Lechonk',
 'Oinkologne',
 'Tarountula',
 'Spidops',
 'Nymble',
 'Lokix',
 'Pawmi',
 'Pawmo',
 'Pawmot',
 'Tandemaus',
 'Maushold',
 'Fidough',
 'Dachsbun',
 'Smoliv',
 'Dolliv',
 'Arboliva',
 'Squawkabilly',
 'Nacli',
 'Naclstack',
 'Garganacl',
 'Charcadet',
 'Armarouge',
 'Ceruledge',
 'Tadbulb',
 'Bellibolt',
 'Wattrel',
 'Kilowattrel',
 'Maschiff',
 'Mabosstiff',
 'Shroodle',
 'Grafaiai',
 'Bramblin',
 'Brambleghast',
 'Toedscool',
 'Toedscruel',
 'Klawf',
 'Capsakid',
 'Scovillain',
 'Rellor',
 'Rabsca',
 'Flittle',
 'Espathra',
 'Tinkatink',
 'Tinkatuff',
 'Tinkaton',
 'Wiglett',
 'Wugtrio',
 'Bombirdier',
 'Finizen',
 'Palafin',
 'Varoom',
 'Revavroom',
 'Cyclizar',
 'Orthworm',
 'Glimmet',
 'Glimmora',
 'Greavard',
 'Houndstone',
 'Flamigo',
 'Cetoddle',
 'Cetitan',
 'Veluza',
 'Dondozo',
 'Tatsugiri',
 'Annihilape',
 'Clodsire',
 'Farigiraf',
 'Dudunspar

In [210]:
def get_all_pages():
    urls =[]

    for k in range(len(pokemon_names)):
        k = f"https://www.pokepedia.fr/{pokemon_names[k]}"
        urls.append(k)
    return urls

urls = get_all_pages()

In [211]:
def get_pokemon_stats(pokemon_name):
    url = f"https://www.pokepedia.fr/{pokemon_name}"

    response = requests.get(url)

    if response.status_code != 200:
        print(f"Pas d'accès possible pour {pokemon_name}")
        print(response.status_code)
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    index = pokemon_names.index(pokemon_name)


    # Trouver le tableau des statistiques
    stats_table = soup.find("table", class_=f"tableaustandard {pokemon_types[index][0].lower()} tableau-overflow")
    if not stats_table:
        print(f"Pas de tableau de stats trouvé pour {pokemon_name}")
        return None

    # Extraire les statistiques (PV, Attaque, Défense, Vitesse)
    rows = stats_table.find_all("tr")

    stats = {"Nom" : pokemon_name}
    stat_names = ["PV", "Attaque", "Attaque Spéciale", "Défense Spéciale", "Défense", "Vitesse"]  # Stats à récupérer

    for row in rows:
        cols = row.find_all("td")

        if len(cols) >= 2:
            stat_label = cols[0].text.strip()  # Texte de la première colonne (nom de la stat)
            stat_value = cols[1].text.strip()  # Valeur de la deuxième colonne

            if "PV" in stat_label:
                stat_label = "PV"
 
            # Vérifier si c'est bien l'une des statistiques recherchées
            if stat_label in stat_names:
                stats[stat_label] = stat_value

    # Vérifier si on a bien trouvé toutes les stats, sinon mettre "N/A"
    for stat in stat_names:
        if stat not in stats:
            stats[stat] = "N/A"

    return stats # Retourner les statistiques sous forme de dictionnaire

# 🔍 Test avec Keunotor
pokemon_name = "Poussacha"
stats = get_pokemon_stats(pokemon_name)

if stats:
    print(f"📊 Statistiques de {pokemon_name} :", stats)

📊 Statistiques de Poussacha : {'Nom': 'Poussacha', 'PV': '40', 'Attaque': '61', 'Défense': '54', 'Attaque Spéciale': '45', 'Défense Spéciale': '45', 'Vitesse': '65'}


In [212]:
import pandas as pd 

df_gen9 = pd.DataFrame({
    "Nom": pokemon_names,
    "Type": pokemon_types,
    "Géneration": ["2"] * len(pokemon_names)
})

stats_list = []
for name in df_gen9["Nom"]:
    stats = get_pokemon_stats(name)  # Récupérer les stats sous forme de dictionnaire
    if stats:
        stats_list.append(stats)

df_stats =pd.DataFrame(stats_list)

df_gen9 = df_gen9.merge(df_stats, on="Nom", how="left")

In [213]:
len(pokemon_names_anglais)

120

In [214]:
df_gen9["Names"] = pokemon_names_anglais
df_gen9


,Nom,Type,Géneration,PV,Attaque,Défense,Attaque Spéciale,Défense Spéciale,Vitesse,Names
0,Poussacha,[Plante],2,40,61,54,45,45,65,Sprigatito
1,Matourgeon,[Plante],2,61,80,63,60,63,83,Floragato
2,Miascarade,"[Plante, Ténèbres]",2,76,110,70,81,70,123,Meowscarada
3,Chochodile,[Feu],2,67,45,59,63,40,36,Fuecoco
4,Crocogril,[Feu],2,81,55,78,90,58,49,Crocalor
...,...,...,...,...,...,...,...,...,...,...
115,Ire-Foudre,"[Électrik, Dragon]",2,125,73,91,137,89,75,Raging Bolt
116,Roc-de-Fer,"[Roche, Psy]",2,90,120,80,68,108,124,Iron Boulder
117,Chef-de-Fer,"[Acier, Psy]",2,90,72,100,122,108,98,Iron Crown
118,Terapagos,[Normal],2,90,65,85,65,85,60,Terapagos


In [215]:
df_gen9 =df_gen9[["Names", "Nom", "Type", "PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse", "Géneration"]]
df_gen9 = df_gen9.rename(columns={"Nom": "Noms", "Géneration": "Génération"})

In [216]:
cols_numeriques = ["PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse"]

df_gen9[cols_numeriques] = df_gen9[cols_numeriques].apply(pd.to_numeric, errors="coerce")

df_gen9["Somme Stats"] = df_gen9[cols_numeriques].sum(axis=1)

df_gen9["Moyenne Statistique"] = (df_gen9["Somme Stats"] / 6).round(2)

In [218]:
df_gen9.loc[df_gen9["Noms"] == "Câblifère", ["PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse"]] = [83, 89, 71, 173, 71, 83]

In [219]:
df_gen9.loc[df_gen9["Noms"] == "Câblifère", ["Somme Stats", "Moyenne Statistique"]] = [570, 95.0]

In [220]:
colonnes_a_convertir = ["PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse"]  # Liste des colonnes à convertir
df_gen9[colonnes_a_convertir] = df_gen9[colonnes_a_convertir].astype(int)


In [221]:
df_gen9.to_csv("pokemon_gen9.csv", encoding="utf-8")

In [222]:
pokemon_all_gen = pd.read_csv("Data/pokemon_all_gen.csv")
pokemon_all_gen

,Unnamed: 0.1,Unnamed: 0,Names,Noms,Type,PV,Attaque,Attaque Spéciale,Défense,Défense Spéciale,Vitesse,Génération,Somme Stats,Moyenne Statistique
0,0,0,Bulbasaur,Bulbizarre,"['Plante', 'Poison']",45,49,65,49,65,45,1,318.0,53.00
1,1,1,Ivysaur,Herbizarre,"['Plante', 'Poison']",60,62,80,63,80,60,1,405.0,67.50
2,2,2,Venusaur,Florizarre,"['Plante', 'Poison']",80,82,100,83,100,80,1,525.0,87.50
3,3,3,Charmander,Salamèche,['Feu'],39,52,60,43,50,65,1,309.0,51.50
4,4,4,Charmeleon,Reptincel,['Feu'],58,64,80,58,65,80,1,405.0,67.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1020,115,Raging Bolt,Ire-Foudre,"['Électrik', 'Dragon']",125,73,137,91,89,75,2,590.0,98.33
1021,1021,116,Iron Boulder,Roc-de-Fer,"['Roche', 'Psy']",90,120,68,80,108,124,2,590.0,98.33
1022,1022,117,Iron Crown,Chef-de-Fer,"['Acier', 'Psy']",90,72,122,100,108,98,2,590.0,98.33
1023,1023,118,Terapagos,Terapagos,['Normal'],90,65,65,85,85,60,2,450.0,75.00


In [ ]:
pokemon_all_gen[""]